# Example for utilization of powerfactory-tools -- Control

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt
import logging
import pathlib

## Select an compatible PowerFactory Interface
- Import the PowerFactoryInterface (in this example: that is compatible with PowerFactory in version 2024)
- Specify additional PowerFactory configuration
- Name the PowerFactory project

In [ ]:
from powerfactory_tools.utils.io import FileType
from powerfactory_tools.versions.pf2024 import PowerFactoryInterface
from powerfactory_tools.versions.pf2024.constants import NAME_SEPARATOR
from powerfactory_tools.versions.pf2024.interface import ValidPythonVersion
from powerfactory_tools.versions.pf2024.types import CalculationType
from powerfactory_tools.versions.pf2024.types import ModeInpLoad
from powerfactory_tools.versions.pf2024.types import PFClassId
from powerfactory_tools.versions.pf2024.types import ResultExportMode
from powerfactory_tools.versions.pf2024.utils.io import ExportHandler

# PowerFactory configuration
PF_SERVICE_PACK = 2  # mandatory
PF_USER_PROFILE = ""  # specification may be necessary
PF_INI_NAME = ""  # optional specification of ini file name to switch to full version (e.g. PowerFactoryFull for file PowerFactoryFull.ini)
PF_PYTHON_VERSION = ValidPythonVersion.VERSION_3_12  # python version of local code environment must match the python version of PowerFactory API


# Consider to use raw strings to avoid misinterpretation of special characters, e.g. r"dir\New Project" or r"dir\1-HV-grid".
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
EXPORT_PATH = pathlib.Path("control_action_results")

### Define control routine for 3_Bus example grid to run later in the control example 

In [ ]:
def run_three_bus_control_example(pfi: PowerFactoryInterface, export_path: pathlib.Path) -> None:
    """A simple collection of possible control actions.

    This example is related to the provided project "PF2024_PowerFactory-Tools.pfd" with the 3-Bus grid "HV_3_Bus".

    Args:
        pfi {PowerFactoryInterface} -- : an instance of PowerFactoryInterface
        export_path {pathlib.Path} -- directory within to export results
    """
    study_case_name = "3_Bus"
    grid_name = "HV_3_Bus"

    study_case = pfi.switch_study_case(study_case_name)  # noqa: F841

    ## From active study case: get all relevant! terminals, lines, loads and generators
    logging.info("3_Bus : From active study case: get all terminals, lines, loads and generators ...")
    terminals = pfi.terminals(calc_relevant=True)
    lines = pfi.lines(calc_relevant=True)  # noqa: F841
    loads = pfi.loads(calc_relevant=True)
    generators = pfi.generators(calc_relevant=True)  # noqa: F841

    ## Let's dive a bit deeper into the PowerFactoryInterface
    # Get all calculation relevant generators from the active study case, but exclude generators that are out of service
    generators2 = pfi.generators(calc_relevant=True, include_out_of_service=False)  # noqa: F841
    # Get all calculation relevant generators from the active study case, but only the ones from the grid with the specified name
    # As within this study case only one grid is active, the following statement leads to the same result as in variable generators:
    generators3 = pfi.generators(grid_name=grid_name)  # noqa: F841

    # In general, one can use grid_elements() function to get elements by user defined filter, e.g. get again all generators
    generators4 = pfi.grid_elements(  # noqa: F841
        class_name=PFClassId.GENERATOR.value,  # class name, same as passing the raw string: "ElmGenstat"
        name="*",  # name doesn't matter
        grid_name=grid_name,  # which grid is to be used to search for generators
        calc_relevant=True,  # only get calc relevant generators
        include_out_of_service=True,  # include also out of service generators
    )

    ## Raise the consumed power of the load by 5 %
    logging.info("3_Bus : Raise the consumed power of the load by 5 % ...")
    # Change power of loads
    for load in loads:
        load.plini = load.plini * 1.05  # active power in MW
        load.qlini = load.qlini * 1.05  # reactive power in MW

    ## Run load flow
    logging.info("3_Bus : Run load flow and export nodal voltages ...")
    pfi.run_ldf(ac=True, symmetrical=True)

    ## Do uncomment for demonstration purposes: show the PF application window in non-interavtive mode
    # do not close the window by your own by clicking on the red cross, but process via the user input
    # pfi.app.Show()
    # time.sleep(5)   # wait for 5 seconds
    # input("Press Enter to continue...")  # Wait for user input before proceeding
    # pfi.app.Hide()

    ## Export nodal voltages
    # Do further user specific work and fill result_data dictionary based on PF result
    data = {}
    for term in terminals:
        entry = {f"{term.loc_name}__Uabs1": term.GetAttribute("m:Ul")}
        data.update(entry)
    # Store results
    eh = ExportHandler(directory_path=export_path)
    eh.export_user_data(
        data,
        file_type=FileType.CSV,
        file_name="3_bus_nodal_voltages_case_a",
    )

    ## Create new operating case (scenario) and set "Line_2_3" out of service
    logging.info("3_Bus : Create new operating case (scenario) and set 'Line_2_3' out of service ...")
    scenario = pfi.create_scenario(name="op_case_b")
    # Example of using the PowerFactoryInterface manually to create a new object:
    # scenario = pfi.create_object(name="op_case_b", class_name=PFClassId.SCENARIO.value, location=pfi.scenario_dir)  # noqa: ERA001
    if scenario is not None:
        pfi.activate_scenario(scenario)
        # set "Line_2_3" out of service
        line_2_3 = pfi.line("Line_2_3", grid_name=grid_name)
        if line_2_3 is not None:
            line_2_3.outserv = True
        else:
            logging.info("3_Bus : Could not found line 'Line_2_3'.")

    ## Run load flow again and export nodal voltages
    logging.info("3_Bus : Run load flow II and export nodal voltages ...")
    pfi.run_ldf(ac=True, symmetrical=True)
    # Do further user specific work and fill result_data dictionary based on PF result
    data = {}
    for term in terminals:
        entry = {f"{term.loc_name}__Uabs1": term.GetAttribute("m:Ul")}
        data.update(entry)
    # Store results
    eh.export_user_data(
        data,
        file_type=FileType.CSV,
        file_name="3_bus_nodal_voltages_case_b",
    )

### Define control routine for 9_Bus example grid to run later in the control example 

In [ ]:
## Define element variables to monitor for later
# symmetrical load flow case
element_vars_sym = {
    "node": {
        "m:U": "Uabs_PH_E",
        "m:phiu": "Phase_Voltage",
    },
}
# asymmetrical load flow case
element_vars_unsym = {
    "node": {
        "m:U1": "Uabs1",
        "m:U2": "Uabs2",
        "m:U0": "Uabs0",
        "m:phiu1": "Uang1",
        "m:phiu2": "Uang2",
        "m:phiu0": "Uang0",
    },
    "node_reduced": {
        "m:U1": "Uabs1",
        "m:U2": "Uabs2",
        "m:U0": "Uabs0",
    },
    "line": {
        "m:I1:bus1": "Iabs1",
        "m:I2:bus1": "Iabs2",
        "m:I0:bus1": "Iabs0",
        "m:phii1:bus1": "Iang1",
        "m:phii2:bus1": "Iang2",
        "m:phii0:bus1": "Iang0",
    },
}

In [ ]:

def run_nine_bus_control_example(pfi: PowerFactoryInterface, export_path: pathlib.Path) -> None:  # noqa: PLR0915
    """A sophisticated collection of possible control actions.

    This example is related to the provided project "PF2024_PowerFactory-Tools.pfd" with the 9-Bus grid "HV_9_Bus".

    Args:
        pfi {PowerFactoryInterface} -- : an instance of PowerFactoryInterface
        export_path {pathlib.Path} -- directory within to export results
    """
    # select study case "Outage" to see the different results of element selection in example 2
    default_study_case_name = "Base"
    grid_name = "HV_9_Bus"

    ############
    ## Example 1: Request elements of grid
    logging.info("Start control example, section I...")
    # Activate study case "Base"
    study_case = pfi.switch_study_case(default_study_case_name)
    # Create new variant to work within (add second if one already exists with same name)
    variant_folder = pfi.create_folder(name="user_variants", location=pfi.grid_variant_dir, update=True)
    grid_variant = pfi.create_grid_variant(name="control_example", location=variant_folder, force=True)
    if grid_variant:
        pfi.switch_grid_variant(grid_variant.loc_name)
    # Get grid object with specified name
    grid = pfi.grid(grid_name)
    # Get all objects from specific grid
    data = pfi.compile_powerfactory_data(grid)
    # All nodes in grid {grid_name}, also these that are out of service
    terminals_grid = data.terminals

    ############
    ## Example 2: Select special elements
    logging.info("Start control example, section II...")
    # All nodes from the project (source may be multiple grids)
    # --> from all grids (independent if active or not) + all nodes (independent if out of service or not)
    terminals_project = pfi.terminals()  # noqa: F841
    # All nodes from the active study case
    # --> therefore calculation relevant
    terminals_study_case = pfi.terminals(calc_relevant=True)  # noqa: F841
    # As within this study case only one grid is active, the following statement leads to the same result:
    terminals_grid = pfi.terminals(grid_name=grid_name)  # noqa: F841
    # All active nodes from the active study case
    # --> therefore exclude nodes which are not in service
    active_terminals_study_case = pfi.terminals(calc_relevant=True, include_out_of_service=False)

    # Select only terminals with nominal voltage of xx kV
    voltage_threshold = 110
    terminals_sel = []  # selected terminals
    for term in active_terminals_study_case:
        # nominal voltage
        u_n = term.uknom
        if u_n == voltage_threshold:
            terminals_sel.append(term)

    ############
    ## Example 3: Change attribute values
    logging.info("Start control example, section III...")
    # Select some loads that names start with "Load", do not consider LV and MV Loads
    loads = pfi.loads("Load*", grid_name=grid_name)

    # Change power of loads
    for load in loads:
        # for primitive types (int, float, str, bool), value can be assigned directly
        load.plini = 2  # active power in MW
        load.mode_inp = ModeInpLoad.PQ.value  # type: ignore[assignment]
        # need to be fixed: for elaborated types, value ought be assigned using the update_value method of the interface
        # e. g. specify the type of input mask for load power (here, define P and Q)

    ############
    ## Example 4: Create variable monitors (a selection of monitored variables for specific elements)
    logging.info("Start control example, section IV...")
    ## Adapt default result object
    # Get existing default result object ("Alle Berechnungsarten" or "All calculations") -  may not yet exist when first executed
    default_result = pfi.result("All*", study_case_name=study_case.loc_name)
    # Create variable monitor objects for default result
    if default_result is not None:
        for term in terminals_sel:
            # Create variable monitor (unsymmetric case) for each terminal
            pfi.create_variable_monitor(
                element=term,
                result=default_result,
                variables=element_vars_unsym["node"].keys(),
            )
            # Create variable monitor (symmetric case) for each terminal
            pfi.create_variable_monitor(element=term, result=default_result, variables=element_vars_sym["node"].keys())

    ## Create new result object
    # Create new result object
    new_result = pfi.create_result(name="New Results", study_case=study_case)
    new_result.calTp = CalculationType.ALL_CALCULATIONS.value  # type: ignore[assignment]   # would be also the default value

    # Create variable monitor objects
    for term in terminals_sel:
        # Create variable monitor (unsymmetric case) for each terminal
        pfi.create_variable_monitor(
            element=term,
            result=new_result,
            variables=element_vars_unsym["node_reduced"].keys(),
        )
        # Create variable monitor (symmetric case) for each terminal
        pfi.create_variable_monitor(element=term, result=default_result, variables=element_vars_sym["node"].keys())

    ############
    ## Example 5: Run load flow and export results
    logging.info("Start control example, section V...")
    # The results would be stored in the related result objects, which are the $default_result and the $new_result
    ## Run symmetrical AC load flow
    pfi.run_ldf(ac=True, symmetrical=False)

    ## Setup result export - variant I
    # a) Assign variable monitors to a result
    pfi.write_variable_monitors_for_result(default_result)

    # b) Create result export command and assign the result objectand execute
    res_exp_cmd_1 = pfi.create_result_export_command(
        result=default_result,
        study_case=study_case,
        export_path=export_path,
        export_mode=ResultExportMode.CSV,
        name="My_Result_Export",
        file_name="LDF_Results_full",
    )
    if res_exp_cmd_1 is not None:
        # Use english separators for CSV
        res_exp_cmd_1.iopt_sep = False
        res_exp_cmd_1.col_Sep = ","
        res_exp_cmd_1.dec_Sep = "."
        # Execute result export - variant I
        pfi.run_result_export(res_exp_cmd_1)

    ## Setup result export - variant II
    # a) Assign variable monitors to a result
    pfi.write_variable_monitors_for_result(new_result)

    # b) Create result export command and execute
    res_exp_cmd_2 = pfi.create_result_export_command(
        result=new_result,
        study_case=study_case,
        export_path=export_path,
        export_mode=ResultExportMode.CSV,
        name="My_Result_Export_2",
        file_name="LDF_Results_selected",
    )
    if res_exp_cmd_2 is not None:
        # Use english separators for CSV
        res_exp_cmd_2.iopt_sep = False
        res_exp_cmd_2.col_Sep = ","
        res_exp_cmd_2.dec_Sep = "."

        # Execute result export - variant II
        pfi.run_result_export(res_exp_cmd_2)

    ## Export results - Variant III
    # Do further user specific work and fill result_data dictionary based on PF result
    data = {
        f"{terminals_sel[0].loc_name}": {
            "Uabs1": {
                "value": terminals_sel[0].GetAttribute("m:U1"),
            },
        },
    }
    # Store results
    eh = ExportHandler(directory_path=export_path)
    eh.export_user_data(
        data,
        file_type=FileType.JSON,
        file_name=pfi.app.GetActiveStudyCase().loc_name + NAME_SEPARATOR + "custom_user_data",
    )

    ###########
    ## Example 6: Run time domain simulations (RMS, EMT)
    logging.info("Start control example, section VI...")
    sim_length = 3  # in seconds
    # define additional simulation properties: see selection of attributes of PFType.CommandTimeSimulationStart
    rms_sim_data = {"dtgrd": 0.005}  # step size in seconds
    rms_result = pfi.run_rms_simulation(sim_length, data=rms_sim_data)  # symmetrical by default

    # if rms simulation was successful, run result export using builtin function
    if rms_result:
        result_export_data = {"iopt_sep": False, "col_Sep": ",", "dec_Sep": "."}  # Use english separators for CSV
        res_exp_cmd = pfi.create_result_export_command(
            result=rms_result,
            study_case=study_case,
            export_path=export_path,
            export_mode=ResultExportMode.CSV,
            file_name="RMS_Results",
            name="RMS_Export",
            data=result_export_data,
        )
        # Use english separators for CSV
        pfi.run_result_export(res_exp_cmd)

    # Run EMT simulation
    emt_result = pfi.run_emt_simulation(sim_length)  # noqa: F841

    ############
    ## Example 7: Request study cases, operation scenarios and network variants
    logging.info("Start control example, section VII...")
    # Study Case
    study_cases = pfi.study_cases()  # get all  # noqa: F841
    study_case_active = pfi.study_case(only_active=True)  # get only active one  # noqa: F841

    # Network Variation
    variant = pfi.grid_variants()  # get all
    variants_active = pfi.grid_variants(only_active=True)  # get only active one(s)  # noqa: F841

    # Operation Scenarios
    scenarios = pfi.scenarios()  # get all  # noqa: F841
    scenario_active = pfi.scenario(only_active=True)  # get only active one

    ############
    ## Example 8: Create new grid variant, activate it, then change topology within and deactivate it again
    logging.info("Start control example, section VIII...")
    # Create Grid Variant
    variant = pfi.create_grid_variant(name="Variant1", location=variant_folder, update=True)

    # Switch to this new grid variant (activate it and only it)
    pfi.switch_grid_variant(variant.loc_name)

    # Set transformer out of service
    # Deactivate active scenario
    if scenario_active is not None:
        pfi.deactivate_scenario(scenario_active)
    # As no scenario is active anylonger, changes regarding operation are directly saved in grid variant
    # Set transformer out of service
    transformer = pfi.transformer_2w("Transformer_2w_110/20", grid_name=grid_name)
    transformer.outserv = 1

    # Deactivate all grid variants
    for variant in pfi.grid_variants(only_active=True):
        pfi.deactivate_grid_variant(variant)

    ############
    ## Example 9: Create new study case and define related grids and grid variants
    logging.info("Start control example, section IX...")
    study_case = pfi.create_study_case(
        name="Industry_Park_v2",
        grids=pfi.independent_grids(),
        grid_variants=[variant],
        target_datetime=dt.datetime(1980, 1, 1, tzinfo=dt.timezone.utc),
    )
    # Switch to this new study case
    pfi.switch_study_case(study_case.loc_name)

    # collect all actives grids
    grids_active = pfi.grids(calc_relevant=True)  # noqa: F841

    # Let only one grid be active
    pfi.deactivate_grids()
    pfi.activate_grid(pfi.grid(grid_name))

## Execute control action using a controller instance

In [ ]:
_project_name = PROJECT_NAME.split("\\")
full_export_path = pathlib.Path().cwd() / EXPORT_PATH / _project_name[-1]

# Configure logging to output to the notebook's standard output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

with PowerFactoryInterface(
    powerfactory_service_pack=PF_SERVICE_PACK,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PF_PYTHON_VERSION,
    project_name=PROJECT_NAME,
    logging_level=logging.INFO,
    # log_file_path=full_export_path / pathlib.Path("pf_control.log"),  # noqa: ERA001
) as pfi:

    logging.info("3_Bus : Run control example ... ")
    run_three_bus_control_example(pfi, full_export_path)
    logging.info("3_Bus : Run control example ... Done")

    logging.info("9_Bus : Run control example ... ")
    run_nine_bus_control_example(pfi, full_export_path)
    logging.info("9_Bus : Run control example ... Done")

## [Optional] Control using the control function running in a new process with default parameters

In [ ]:
import multiprocessing

from powerfactory_tools.versions.pf2024.interface import DEFAULT_POWERFACTORY_PATH
from powerfactory_tools.versions.pf2024.interface import DEFAULT_PYTHON_VERSION
from powerfactory_tools.versions.pf2024.interface import ValidPythonVersion

# Define a controller process
class PowerFactoryControllerProcess(multiprocessing.Process):
    def __init__(
        self,
        *,
        export_path: pathlib.Path,
        project_name: str,
        powerfactory_path: pathlib.Path = DEFAULT_POWERFACTORY_PATH,
        powerfactory_service_pack: int = "",
        powerfactory_user_profile: str = "",
        powerfactory_ini_name: str = "",
        python_version: ValidPythonVersion = DEFAULT_PYTHON_VERSION,
        logging_level: int = logging.DEBUG,
        log_file_path: pathlib.Path | None = None,
    ) -> None:
        super().__init__()
        self.export_path = export_path
        self.project_name = project_name
        self.powerfactory_path = powerfactory_path
        self.powerfactory_service_pack = powerfactory_service_pack
        self.powerfactory_user_profile = powerfactory_user_profile
        self.powerfactory_ini_name = powerfactory_ini_name
        self.python_version = python_version
        self.logging_level = logging_level
        self.log_file_path = log_file_path

    def run(self) -> None:
        pfi = PowerFactoryInterface(
            project_name=self.project_name,
            powerfactory_service_pack=self.powerfactory_service_pack,
            powerfactory_user_profile=self.powerfactory_user_profile,
            powerfactory_ini_name=self.powerfactory_ini_name,
            python_version=self.python_version,
            logging_level=self.logging_level,
            log_file_path=self.log_file_path,
        )
        run_nine_bus_control_example(pfi, self.export_path)

### As the control function is executed in a process that is terminated after execution, the PowerFactory API is also closed.

In [ ]:
_project_name = PROJECT_NAME.split("\\")
full_export_path = pathlib.Path().cwd() / EXPORT_PATH / _project_name[-1]


# Initialize controller process
process = PowerFactoryControllerProcess(
    powerfactory_service_pack=PF_SERVICE_PACK,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=DEFAULT_PYTHON_VERSION,
    project_name=PROJECT_NAME,
    export_path=full_export_path,
    logging_level=logging.INFO,
    log_file_path=full_export_path / pathlib.Path("pf_control.log"),
)
# Run process
process.start()
process.join()